In [1]:
# define rooms and items

# define game room

couch = {
    "name": "couch",
    "type": "furniture",
}

door_a = {
    "name": "door a",
    "type": "door",
    "objects": [],
}

key_a = {
    "name": "key for door a",
    "type": "key",
    "target": door_a,
    "objects": [],
}

piano = {
    "name": "piano",
    "type": "furniture",
}

game_room = {
    "name": "game room",
    "type": "room",
    "objects": [],
}

# define bedroom 1

queen_bed = {
    "name": "queen bed",
    "type": "furniture",
}

door_b = {
    "name": "door b",
    "type": "door",
    "objects": [],
}

key_b = {
    "name": "key for door b",
    "type": "key",
    "target": door_b,
    "objects": [],
}

door_c = {
    "name": "door c",
    "type": "door",
    "objects": [],
}

bedroom_1 = {
    "name": "bedroom 1",
    "type": "room",
    "objects": [],
}

# define living room

dining_table = {
    "name": "dining table",
    "type": "furniture",
}

door_d = {
    "name": "door d",
    "type": "door",
    "objects": [],
}

living_room = {
    "name": "living room",
    "type": "room",
    "objects": [],
}

cookye = {
    "name": "COOKYE",
    "type": "key",
    "target": "dining table",
    "objects": [],
}

# bedroom 2

double_bed = {
    "name": "double bed",
    "type": "furniture",
}

dresser = {
    "name": "dresser",
    "type": "furniture",
}


key_c = {
    "name": "key for door c",
    "type": "key",
    "target": door_c,
    "objects": [],
}

key_d = {
    "name": "key for door d",
    "type": "key",
    "target": door_d,
    "objects": [],
}

bedroom_2 = {
    "name": "bedroom 2",
    "type": "room",
    "objects": [],
}


# define trap door

trap_door = {
    "name": "secret door",
    "type": "door",
    "objects": [],
}

key_trap_door = {
    "name": "secret kye",
    "type": "key",
    "target": trap_door,
    "objects": [],
}


# define outside

outside = {
  "name": "outside"
}

all_rooms = [game_room, bedroom_1, bedroom_2, living_room, outside]

all_doors = [door_a, door_b, door_c, door_d, trap_door]


# define which items/rooms are related

object_relations = {
    "game room": [couch, piano, door_a],
    "couch": [key_trap_door],
    "piano": [key_a],
    "outside": [door_d],
    "door a": [game_room, bedroom_1],
    "door b": [bedroom_1, bedroom_2],
    "bedroom 1":[queen_bed, door_a, door_b, door_c],
    "queen bed": [key_b],
    "bedroom 2": [door_b, double_bed, dresser, trap_door],
    "double bed": [key_c],
    "dresser": [key_d],
    "living room": [dining_table, door_d],
    "door c": [bedroom_1, living_room],
    "door d": [outside, living_room],
    "secret door": [bedroom_2],
    "dining table": [cookye]
}

# define game state. Do not directly change this dict. 
# Instead, when a new game starts, make a copy of this
# dict and use the copy to store gameplay state. This 
# way you can replay the game multiple times.

INIT_GAME_STATE = {
    "current_room": game_room,
    "keys_collected": [],
    "target_room": outside
}

In [2]:
def linebreak():
    """
    Print a line break
    """
    print("\n\n")

def start_game():
    print("You wake up on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house, NOW!")
    play_room(game_state["current_room"])
    replay = input("Play Again ? (yes/no)").strip()
    if replay == "yes":
        start_game()  # Redémarrez complètement le jeu
    elif replay == "no":
        print("Ok ! Bye Loser.")
        exit()  # Quittez complètement le jeu


def play_room(room):
    """
    Play a room. First check if the room being played is the target room.
    If it is, the game will end with success. Otherwise, let player either 
    explore (list all items in this room) or examine an item found here.
    """
    game_state["current_room"] = room
    if(game_state["current_room"] == game_state["target_room"]):
        print("Congrats! You escaped the room!")
    else:
        print("You are now in " + room["name"])
        intended_action = input("What would you like to do? Type 'explore' or 'examine'?").strip()
        if intended_action == "explore":
            explore_room(room)
            play_room(room)
        elif intended_action == "examine":
            examine_item(input("What would you like to examine?").strip())
        else:
            print("Not sure what you mean. Type 'explore' or 'examine'.")
            play_room(room)
        linebreak()

def explore_room(room):
    """
    Explore a room. List all items belonging to this room.
    """
    items = [i["name"] for i in object_relations[room["name"]]]
    print("You explore the room. This is " + room["name"] + ". You find " + ", ".join(items))

def get_next_room_of_door(door, current_room):
    """
    From object_relations, find the two rooms connected to the given door.
    Return the room that is not the current_room.
    """
    connected_rooms = object_relations[door["name"]]
    return next(room for room in connected_rooms if room != current_room)

def examine_item(item_name):
    current_room = game_state["current_room"]
    next_room = ""
    output = None
    
    for item in object_relations[current_room["name"]]:
        if item["name"] == item_name:
            output = "You examine " + item_name + ". "
            if item["type"] == "door":
                have_key = False
                for key in game_state["keys_collected"]:
                    if key["target"] == item:
                        have_key = True
                if have_key:
                    if item == trap_door:
                        handle_trap_door()  # Appeler la fonction spéciale
                    else:
                        output += "You unlock it with a key you have."
                        next_room = get_next_room_of_door(item, current_room)
                else:
                    output += "It is locked but you don't have the key."
            else:
                if item["name"] in object_relations and len(object_relations[item["name"]]) > 0:
                    item_found = object_relations[item["name"]].pop()
                    game_state["keys_collected"].append(item_found)
                    output += "You find " + item_found["name"] + "."
                else:
                    output += "There isn't anything interesting about it."
            print(output)
            break

    if output is None:
        print("The item you requested is not found in the current room.")
    
    if next_room and input("Do you want to go to the next room? Enter 'yes' or 'no'").strip() == 'yes':
        play_room(next_room)
    else:
        play_room(current_room)
        
def handle_trap_door():
    print("You Lose ! It was a TRAP mouahahah!")
    replay = input("Play Again? (yes/no)").strip()
    if replay == "yes":
        # Conservez les clés collectées
        collected_keys = game_state["keys_collected"]
        game_state["current_room"] = game_room
        
        # Mettez à jour la liste d'objets dans la "game room" avec les clés collectées
        game_room["objects"] = collected_keys
        
        start_game()  # Redémarrez le jeu
        
    elif replay == "no":
        print("YOU CAN NOT ESCAPE ! You HAVE to finish the game you coward !")
        exit()  # Quittez complètement le jeu

In [3]:
game_state = INIT_GAME_STATE.copy()

start_game()

You wake up on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house, NOW!
You are now in game room
What would you like to do? Type 'explore' or 'examine'?examine
What would you like to examine?couch
You examine couch. You find secret kye.
You are now in game room
What would you like to do? Type 'explore' or 'examine'?examine
What would you like to examine?piano
You examine piano. You find key for door a.
You are now in game room
What would you like to do? Type 'explore' or 'examine'?examine
What would you like to examine?door a
You examine door a. You unlock it with a key you have.
Do you want to go to the next room? Enter 'yes' or 'no'yes
You are now in bedroom 1
What would you like to do? Type 'explore' or 'examine'?examine
What would you like to examine?queen bed
You examine queen bed. You find ke

KeyboardInterrupt: Interrupted by user